# Capítulo 09 - Meta Queries e Introspecção

Este notebook explora as capacidades de introspecção do DuckDB para analisar metadados, estruturas e estatísticas de dados.

## 📚 Tópicos Abordados:
1. DESCRIBE (estrutura de tabelas)
2. SUMMARIZE (estatísticas automáticas)
3. Catálogo do Sistema (duckdb_tables, duckdb_columns, etc.)
4. Informações de Configuração
5. Análise de Esquemas
6. Query Profiling

## 🔍 Vantagens:
- **Exploração rápida** de dados desconhecidos
- **Estatísticas automáticas** sem código adicional
- **Catálogo completo** do banco de dados
- **Análise de performance** integrada

## 1. Setup e Preparação

In [ ]:
import duckdb
import pandas as pd
import numpy as np

# Criar conexão
con = duckdb.connect(':memory:')

print(f"DuckDB versão: {duckdb.__version__}")
print("✓ Conexão criada!")

### 1.1 Criar Dados de Exemplo

In [ ]:
# Criar tabela de vendas
con.execute("""
    CREATE TABLE vendas (
        id INTEGER PRIMARY KEY,
        data DATE,
        produto VARCHAR,
        categoria VARCHAR,
        quantidade INTEGER,
        preco DECIMAL(10, 2),
        regiao VARCHAR,
        vendedor VARCHAR
    )
""")

# Inserir dados de exemplo
con.execute("""
    INSERT INTO vendas VALUES
    (1, '2024-01-15', 'Notebook', 'Eletrônicos', 5, 2500.00, 'Sudeste', 'Alice'),
    (2, '2024-01-16', 'Mouse', 'Eletrônicos', 20, 50.00, 'Sul', 'Bob'),
    (3, '2024-01-17', 'Teclado', 'Eletrônicos', 15, 150.00, 'Sudeste', 'Alice'),
    (4, '2024-01-18', 'Monitor', 'Eletrônicos', 8, 800.00, 'Norte', 'Charlie'),
    (5, '2024-01-19', 'Cadeira', 'Móveis', 10, 600.00, 'Sudeste', 'Alice'),
    (6, '2024-01-20', 'Mesa', 'Móveis', 5, 1200.00, 'Sul', 'Bob'),
    (7, '2024-01-21', 'Webcam', 'Eletrônicos', 12, 300.00, 'Norte', 'Charlie'),
    (8, '2024-01-22', 'Headset', 'Eletrônicos', 18, 200.00, 'Sudeste', 'Alice')
""")

print("✓ Tabela 'vendas' criada com 8 registros")

# Visualizar dados
result = con.execute("SELECT * FROM vendas LIMIT 5").fetchdf()
print("\nPrimeiros registros:")
print(result)

## 2. DESCRIBE - Estrutura de Tabelas

### 2.1 DESCRIBE Básico

In [ ]:
# Ver estrutura da tabela
print("Estrutura da tabela 'vendas':")
result = con.execute("DESCRIBE vendas").fetchdf()
print(result)

print("\n💡 DESCRIBE mostra colunas, tipos e nullabilidade")

### 2.2 DESCRIBE de Query

In [ ]:
# Ver estrutura do resultado de uma query
print("Estrutura de uma query:")
result = con.execute("""
    DESCRIBE 
    SELECT 
        categoria,
        SUM(quantidade) as total_quantidade,
        AVG(preco) as preco_medio
    FROM vendas
    GROUP BY categoria
""").fetchdf()
print(result)

print("\n✓ Útil para ver tipos resultantes de agregações!")

## 3. SUMMARIZE - Estatísticas Automáticas

### 3.1 SUMMARIZE Completo

In [ ]:
# Estatísticas automáticas de todas as colunas
print("Estatísticas da tabela 'vendas':")
result = con.execute("SUMMARIZE vendas").fetchdf()
print(result)

print("\n✨ SUMMARIZE gera estatísticas completas automaticamente!")

### 3.2 Análise de Estatísticas

In [ ]:
# Encontrar colunas com alta cardinalidade
print("Análise customizada das estatísticas:")
result = con.execute("""
    SELECT 
        column_name,
        column_type,
        approx_unique,
        count,
        ROUND(approx_unique * 100.0 / count, 2) as cardinalidade_pct
    FROM (SUMMARIZE vendas)
    WHERE approx_unique IS NOT NULL
    ORDER BY cardinalidade_pct DESC
""").fetchdf()
print(result)

print("\n💡 Identifica colunas únicas ou com muitos valores distintos")

### 3.3 Detectar Valores Nulos

In [ ]:
# Verificar colunas com valores nulos
print("Colunas com valores nulos:")
result = con.execute("""
    SELECT 
        column_name,
        count,
        null_percentage
    FROM (SUMMARIZE vendas)
    WHERE null_percentage > 0
    ORDER BY null_percentage DESC
""").fetchdf()

if len(result) > 0:
    print(result)
else:
    print("✓ Nenhuma coluna tem valores nulos!")

## 4. Catálogo do Sistema

### 4.1 Listar Todas as Tabelas

In [ ]:
# Ver todas as tabelas do banco
print("Tabelas no banco de dados:")
result = con.execute("""
    SELECT 
        table_name,
        schema_name,
        estimated_size
    FROM duckdb_tables()
    ORDER BY table_name
""").fetchdf()
print(result)

### 4.2 Listar Todas as Colunas

In [ ]:
# Ver todas as colunas de todas as tabelas
print("Colunas da tabela 'vendas':")
result = con.execute("""
    SELECT 
        column_name,
        data_type,
        is_nullable,
        column_default
    FROM duckdb_columns()
    WHERE table_name = 'vendas'
    ORDER BY column_index
""").fetchdf()
print(result)

### 4.3 Listar Views

In [ ]:
# Criar uma view de exemplo
con.execute("""
    CREATE VIEW vendas_resumo AS
    SELECT 
        categoria,
        COUNT(*) as total_vendas,
        SUM(quantidade * preco) as valor_total
    FROM vendas
    GROUP BY categoria
""")

# Listar views
print("Views no banco de dados:")
result = con.execute("""
    SELECT 
        view_name,
        schema_name,
        sql
    FROM duckdb_views()
""").fetchdf()
print(result)

### 4.4 Listar Índices

In [ ]:
# Criar índice de exemplo
try:
    con.execute("CREATE INDEX idx_categoria ON vendas(categoria)")
    print("✓ Índice criado")
except Exception as e:
    print(f"⚠️ Índice já existe ou erro: {e}")

# Listar índices
print("\nÍndices no banco de dados:")
result = con.execute("""
    SELECT 
        index_name,
        table_name,
        is_unique,
        is_primary
    FROM duckdb_indexes()
""").fetchdf()
print(result)

### 4.5 Listar Schemas

In [ ]:
# Ver todos os schemas
print("Schemas disponíveis:")
result = con.execute("""
    SELECT 
        schema_name,
        sql
    FROM duckdb_schemas()
""").fetchdf()
print(result)

## 5. Configurações do Sistema

### 5.1 Ver Todas as Configurações

In [ ]:
# Listar configurações importantes
print("Configurações do DuckDB:")
result = con.execute("""
    SELECT 
        name,
        value,
        description
    FROM duckdb_settings()
    WHERE name IN (
        'threads',
        'memory_limit',
        'default_order',
        'enable_progress_bar',
        'enable_profiling'
    )
    ORDER BY name
""").fetchdf()
print(result)

### 5.2 Ver Configuração Específica

In [ ]:
# Ver configuração atual de threads
result = con.execute("""
    SELECT value 
    FROM duckdb_settings() 
    WHERE name = 'threads'
""").fetchone()

print(f"Threads configurados: {result[0]}")

# Ver memória
result = con.execute("""
    SELECT value 
    FROM duckdb_settings() 
    WHERE name = 'memory_limit'
""").fetchone()

print(f"Limite de memória: {result[0]}")

### 5.3 Listar Extensões

In [ ]:
# Ver extensões disponíveis e instaladas
print("Extensões do DuckDB:")
result = con.execute("""
    SELECT 
        extension_name,
        loaded,
        installed,
        description
    FROM duckdb_extensions()
    WHERE installed = true OR loaded = true
    ORDER BY extension_name
""").fetchdf()
print(result)

## 6. Query Profiling e Análise

### 6.1 EXPLAIN - Plano de Execução

In [ ]:
# Ver plano de execução
print("Plano de execução:")
result = con.execute("""
    EXPLAIN
    SELECT 
        categoria,
        SUM(quantidade * preco) as valor_total
    FROM vendas
    WHERE regiao = 'Sudeste'
    GROUP BY categoria
    ORDER BY valor_total DESC
""").fetchall()

for linha in result:
    print(linha[1])

print("\n💡 EXPLAIN mostra como a query será executada")

### 6.2 EXPLAIN ANALYZE - Execução Real

In [ ]:
# Ver execução real com timing
print("Execução com timing:")
result = con.execute("""
    EXPLAIN ANALYZE
    SELECT 
        categoria,
        COUNT(*) as total,
        SUM(quantidade * preco) as valor
    FROM vendas
    GROUP BY categoria
""").fetchall()

for linha in result:
    print(linha[1])

print("\n✨ EXPLAIN ANALYZE executa e mostra tempos reais!")

### 6.3 Habilitar Profiling

In [ ]:
# Habilitar profiling
con.execute("SET enable_profiling = 'json'")
con.execute("SET profiling_output = '/tmp/duckdb_profile.json'")

print("✓ Profiling habilitado")
print("Arquivo de saída: /tmp/duckdb_profile.json")

# Executar query
result = con.execute("""
    SELECT 
        vendedor,
        COUNT(*) as total_vendas,
        SUM(quantidade * preco) as valor_total
    FROM vendas
    GROUP BY vendedor
    ORDER BY valor_total DESC
""").fetchdf()

print("\nResultado da query:")
print(result)

# Desabilitar profiling (formato correto)
try:
    con.execute("RESET enable_profiling")
    print("\n✓ Profiling desabilitado")
except:
    print("\n💡 Profiling continuará ativo")

## 7. Análise de Esquema Completa

### 7.1 Relatório Completo de uma Tabela

In [ ]:
# Gerar relatório completo
print("="*60)
print("RELATÓRIO COMPLETO - Tabela 'vendas'")
print("="*60)

# 1. Informações básicas
print("\n1. INFORMAÇÕES BÁSICAS:")
result = con.execute("""
    SELECT 
        COUNT(*) as total_linhas,
        COUNT(DISTINCT id) as ids_unicos
    FROM vendas
""").fetchdf()
print(result)

# 2. Estrutura
print("\n2. ESTRUTURA:")
result = con.execute("DESCRIBE vendas").fetchdf()
print(result)

# 3. Estatísticas
print("\n3. ESTATÍSTICAS:")
result = con.execute("""
    SELECT 
        column_name,
        column_type,
        min,
        max,
        approx_unique,
        avg,
        std
    FROM (SUMMARIZE vendas)
""").fetchdf()
print(result)

# 4. Índices
print("\n4. ÍNDICES:")
result = con.execute("""
    SELECT index_name, is_unique, is_primary
    FROM duckdb_indexes()
    WHERE table_name = 'vendas'
""").fetchdf()
print(result)

print("\n" + "="*60)

## 8. Casos de Uso Práticos

### 8.1 Descobrir Schema de Arquivo Desconhecido

In [ ]:
print("""Explorar arquivo desconhecido:

-- Ver estrutura sem carregar tudo
DESCRIBE SELECT * FROM 'arquivo_desconhecido.csv';

-- Ver primeiras linhas
SELECT * FROM 'arquivo_desconhecido.csv' LIMIT 5;

-- Estatísticas completas
SUMMARIZE SELECT * FROM 'arquivo_desconhecido.csv';

-- Contagem rápida
SELECT COUNT(*) FROM 'arquivo_desconhecido.csv';
""")

print("\n✨ Explore dados sem escrever código!")

### 8.2 Auditoria de Qualidade de Dados

In [ ]:
# Verificação de qualidade
print("Auditoria de qualidade de dados:")
print()

# 1. Colunas com valores nulos
print("1. Verificar valores nulos:")
result = con.execute("""
    SELECT 
        column_name,
        null_percentage
    FROM (SUMMARIZE vendas)
    WHERE null_percentage > 0
""").fetchdf()
if len(result) > 0:
    print(result)
else:
    print("✓ Sem valores nulos")

# 2. Duplicados
print("\n2. Verificar duplicados:")
result = con.execute("""
    SELECT COUNT(*) - COUNT(DISTINCT id) as duplicados
    FROM vendas
""").fetchone()
print(f"Registros duplicados: {result[0]}")

# 3. Valores extremos
print("\n3. Valores extremos:")
result = con.execute("""
    SELECT 
        'preco' as coluna,
        MIN(preco) as minimo,
        MAX(preco) as maximo,
        AVG(preco) as media
    FROM vendas
""").fetchdf()
print(result)

print("\n✓ Auditoria completa!")

### 8.3 Comparar Duas Tabelas

In [ ]:
# Criar segunda tabela para comparação
con.execute("""
    CREATE TABLE vendas_backup AS
    SELECT * FROM vendas
""")

print("Comparar estruturas de tabelas:")
print()

# Comparar número de linhas
print("1. Contagem de linhas:")
result = con.execute("""
    SELECT 
        'vendas' as tabela,
        COUNT(*) as linhas
    FROM vendas
    UNION ALL
    SELECT 
        'vendas_backup' as tabela,
        COUNT(*) as linhas
    FROM vendas_backup
""").fetchdf()
print(result)

# Comparar schemas
print("\n2. Comparar colunas:")
result = con.execute("""
    SELECT 
        v1.column_name,
        v1.data_type as tipo_vendas,
        v2.data_type as tipo_backup
    FROM duckdb_columns() v1
    FULL OUTER JOIN duckdb_columns() v2
        ON v1.column_name = v2.column_name
    WHERE v1.table_name = 'vendas'
      AND v2.table_name = 'vendas_backup'
    ORDER BY v1.column_index
""").fetchdf()
print(result)

print("\n✓ Tabelas comparadas!")

## 🎯 Resumo do Capítulo

### ✅ Comandos Principais:

**1. Introspecção:**
- `DESCRIBE` → Estrutura de tabelas/queries
- `SUMMARIZE` → Estatísticas automáticas
- `EXPLAIN` → Plano de execução
- `EXPLAIN ANALYZE` → Timing real

**2. Catálogo:**
- `duckdb_tables()` → Listar tabelas
- `duckdb_columns()` → Listar colunas
- `duckdb_views()` → Listar views
- `duckdb_indexes()` → Listar índices
- `duckdb_schemas()` → Listar schemas

**3. Configuração:**
- `duckdb_settings()` → Ver configurações
- `duckdb_extensions()` → Ver extensões

### 🔍 Casos de Uso:
1. **Exploração de dados** desconhecidos
2. **Auditoria de qualidade** de dados
3. **Análise de performance** de queries
4. **Documentação automática** de schemas
5. **Comparação de estruturas**

### 💡 Boas Práticas:
- Use `SUMMARIZE` antes de análises
- Use `EXPLAIN` para otimizar queries
- Monitore configurações de performance
- Documente schemas com meta queries
- Automatize auditorias de qualidade

### 🚀 Vantagens:
- ⚡ **Rápido**: Estatísticas em segundos
- 🔍 **Completo**: Informações detalhadas
- 🤖 **Automático**: Sem código adicional
- 📊 **Integrado**: SQL nativo

### 📚 Próximo Capítulo:
Performance e otimizações!